In [112]:
from lib.agents import *
import numpy as np
from queue import PriorityQueue

In [113]:
class Wall(Thing):
    pass

class Goal(Thing):
    pass


In [114]:
from random import choice

class OurAgent(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, loc):
        self.location[0] += loc[0]
        self.location[1] += loc[1]
    
    def turn(self, d):
        self.direction = Direction(d)
        
def program(percepts):
    global it
    '''Returns an action based on it's percepts'''
        
    for p in percepts:
        if isinstance(p, Wall):
            print("Cannot move in a wall\nImplement the algorithm again")
            
    it += 1
    return steps[it]

In [115]:
class Maze(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        for thing in self.list_things_at(agent.location):
            if not isinstance(thing, OurAgent):
                things.append(thing)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'right':
            agent.moveforward([1,0])
        elif action == 'left':
            agent.moveforward([-1,0])
        elif action == 'up':
            agent.moveforward([0,-1])
        elif action == 'down':
            agent.moveforward([0,1])
                    
    def is_done(self):
        things = []
        for agent in self.agents:
            for thing in self.list_things_at(agent.location):
                things.append(thing)
        for thing in things:
            if isinstance(thing, Goal):
                return True
        return False


Now that our maze_instance is ready for the 2D motion of our energetic dog, lets test it!

### Helper Functions

In [116]:
def get_neighbors(s,maze):
    y_len = len(maze)
    x_len = len(maze[0])
    y, x = s
    neighbors = []
    
    # Check Up
    if y > 0 and maze[y - 1][x] != 1:
        neighbors.append((y - 1, x))

    # Check Right
    if x < x_len - 1 and maze[y][x + 1] != 1:
        neighbors.append((y, x + 1))

    # Check Down
    if y < y_len - 1 and maze[y + 1][x] != 1:
        neighbors.append((y + 1, x))

    # Check Left
    if x > 0 and maze[y][x - 1] != 1:
        neighbors.append((y, x - 1))

    return neighbors

def get_relative_move(s, d):
    dy = d[0] - s[0]
    dx = d[1] - s[1]

    if dy == -1 and dx == 0:
        return "up"
    elif dy == 0 and dx == 1:
        return "right"
    elif dy == 1 and dx == 0:
        return "down"
    elif dy == 0 and dx == -1:
        return "left"
    else:
        return None

def get_cost(current,neighbor,cost_uptill_now=0):
    dy = neighbor[0] - current[0]
    dx = neighbor[1] - current[1]
    cost = 0
    if dy == -1 and dx == 0:
        cost = 2
    elif dy == 0 and dx == 1:
        cost = 4
    elif dy == 1 and dx == 0:
        cost = 3
    elif dy == 0 and dx == -1:
        cost = 1
    else:
        cost = 0
    return cost + cost_uptill_now  
  
def get_heuristic(start,goal):
    current_position = np.array(start)
    goal_position = np.array(goal)
    return np.sum(np.abs(goal_position-current_position))

def get_evaluation(current,neighbor,goal,cost_uptill_now):
    return get_cost(current,neighbor),cost_uptill_now + get_heuristic(current,goal)

## BFS

In [117]:
def bfs(maze):
    expanded_node=0
    fringe_node=0
    path_length=0
    s = tuple(np.argwhere(maze == 3)[0])
    queue = [s]
    visited = {s: None}  # Use to keep track of previous position

    while queue:
        position = queue.pop(0)
        expanded_node+=1
        if maze[position] == 2:
            # Backtrack from the goal to the start to get the path
            path = []
            while position is not None:
                if visited[position] is not None:
                    path.append(get_relative_move(visited[position], position))
                position = visited[position]
            path_length+=len(path)
            return {
                "expanded_nodes":expanded_node,
                "fringe_nodes":fringe_node,
                "path_length":path_length,
                "path":path[::-1]
                 } # Reverse the path
        else:
            neighbors = get_neighbors(position,maze)
            for neighbor in neighbors:
                if neighbor not in visited:
                    queue.append(neighbor)
                    visited[neighbor] = position  # Set previous of path to the neighbor
                fringe_node=max(fringe_node,len(queue))
    return []

## DFS

In [118]:
def dfs(maze):
    expanded_node=0
    fringe_node=0
    path_length=0
    s = tuple(np.argwhere(maze == 3)[0])
    queue = [s]
    visited = {s: None}  # Use to keep track of previous position

    while queue:
        position = queue.pop()
        expanded_node+=1
        if maze[position] == 2:
            # Backtrack from the goal to the start to get the path
            path = []
            while position is not None:
                if visited[position] is not None:
                    path.append(get_relative_move(visited[position], position))
                position = visited[position]
            path_length+=len(path)
            return {
                "expanded_nodes":expanded_node,
                "fringe_nodes":fringe_node,
                "path_length":path_length,
                "path":path[::-1]
                 }  # Reverse the path
        else:
            neighbors = get_neighbors(position,maze)
            for neighbor in neighbors:
                if neighbor not in visited:
                    queue.append(neighbor)
                    visited[neighbor] = position  # Set previous of path to the neighbor
                fringe_node=max(fringe_node,len(queue))

    return []

# UCS

In [119]:
def UCS(maze):
    expanded_node=0
    fringe_node=0
    path_length=0
    queue = PriorityQueue()
    start = tuple(np.argwhere(maze == 3)[0])
    goal = tuple(np.argwhere(maze == 2)[0])
    queue.put((0, start)) 
    visited = {start: None}
    g_cost = {start: 0}
    path = []
    path.append(start)
    
    while not queue.empty():
        _, current_location = queue.get()
        expanded_node+=1
        path.append(current_location)
        
        if current_location == goal:
            while current_location is not None:
                if visited[current_location] is not None:
                    path.append(get_relative_move(visited[current_location], current_location))
                current_location = visited[current_location]
            path_length+=len(path)
            return {
                "expanded_nodes":expanded_node,
                "fringe_nodes":fringe_node,
                "path_length":path_length,
                "path":path[::-1]
                 }  # Reverse the path  
        else:
            neighbors = get_neighbors(current_location, maze)

            for neighbor in neighbors:
                new_g_cost = get_cost(current_location, neighbor,g_cost[current_location])
                if neighbor not in g_cost or new_g_cost < g_cost[neighbor]:
                    g_cost[neighbor] = new_g_cost
                    f_cost = new_g_cost + get_heuristic(neighbor, goal)
                    queue.put((f_cost, neighbor)) 
                    visited[neighbor] = current_location
                fringe_node=max(fringe_node,queue.qsize())
    return []


# Greedy BFS

In [120]:
def Best_firstSearch(maze):
    expanded_node=0
    fringe_node=0
    path_length=0
    queue = PriorityQueue()
    start = tuple(np.argwhere(maze == 3)[0])
    goal = tuple(np.argwhere(maze == 2)[0])
    queue.put((0, start)) 
    visited = {start: None}
    path = []
    path.append(start)
    while not queue.empty():
        _, current_location = queue.get()
        expanded_node+=1
        path.append(current_location)
        if current_location == goal:
            while current_location is not None:
                if visited[current_location] is not None:
                    path.append(get_relative_move(visited[current_location], current_location))
                current_location = visited[current_location]
            path_length+=len(path)
            return {
                "expanded_nodes":expanded_node,
                "fringe_nodes":fringe_node,
                "path_length":path_length,
                "path":path[::-1]
                 }  # Reverse the path  
        neighbors = get_neighbors(current_location, maze)
        for neighbor in neighbors:
            if neighbor not in visited:
                queue.put((get_heuristic(neighbor, goal), neighbor))
                visited[neighbor] = current_location
            fringe_node=max(fringe_node,queue.qsize())
    return [] 

## AStar

In [121]:
def astar(maze):
    expanded_node=0
    fringe_node=0
    path_length=0
    queue = PriorityQueue()
    s = tuple(np.argwhere(maze == 3)[0])
    goal = tuple(np.argwhere(maze == 2)[0])
    queue.put((0, s))
    visited = {s: None}  
    g_cost = {s: 0}

    while not queue.empty():
        _, current = queue.get()
        expanded_node+=1
        if maze[current] == 2:
            path = []
            while current is not None:# backtracking
                if visited[current] is not None:
                    path.append(get_relative_move(visited[current], current))
                current = visited[current]
            path_length+=len(path)
            return {
                "expanded_nodes":expanded_node,
                "fringe_nodes":fringe_node,
                "path_length":path_length,
                "path":path[::-1]
                 }  # Reverse the path 
        neighbors = get_neighbors(current, maze)
        for neighbor in neighbors:
            new_g_cost = get_cost(current, neighbor, g_cost[current])
            if neighbor not in g_cost or new_g_cost < g_cost[neighbor]:
                g_cost[neighbor] = new_g_cost
                f_cost = get_evaluation(current, neighbor, goal, new_g_cost)
                queue.put((f_cost, neighbor))
                visited[neighbor] = current  # set previous of path to the neighbor
            fringe_node=max(fringe_node,queue.qsize())

    return []

In [122]:
maze_layout = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
    [1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]


maze_instance = Maze(20, 20, color={'Goal': (0, 128, 0), 'Wall': (165,42,42), 'OurAgent': (255, 255, 0)})

def algo(maze):
    """
    implement this function using the maze from input
    3 in the maze represents the location of the agent
    2 represents the goal
    1 represents the wall
    0 represent the free space in which our agent can walk
    return and array of moves like this e.g. ["down", "right", "up", "left", "left", "down"]
    so that the agent will be at the goal 
    use maze_instance.run(0) to show updates without running logic
    """
    # return bfs(maze)
    # return dfs(maze)
    # return astar(maze)    
    return astar(maze)
    # return Best_firstSearch(maze)

# Add walls to the maze_instance
for i in range(len(maze_layout)):
    for j in range(len(maze_layout[0])):
        if maze_layout[i][j] == 1:
            maze_instance.add_thing(Wall(), [i, j])


agent = OurAgent(program)
goal = Goal()


start = input("Enter the coordinates of the start node (x, y): ")
end = input("Enter the coordinates of the goal node (x, y): ")

start = [int(coord) for coord in start.split(",")]
end = [int(coord) for coord in end.split(",")]

maze_layout[start[1]][start[0]] = 3
maze_layout[end[1]][end[0]] = 2    


maze_instance.add_thing(agent, start)
maze_instance.add_thing(goal, end)
maze_instance.exogenous_change()
it = -1
            
maze_instance.delete_thing(agent)
maze_instance.delete_thing(goal)



dic = algo(np.transpose(maze_layout))
steps=dic['path']
path_length=dic['path_length']
time=dic['expanded_nodes']
space=dic['fringe_nodes']
maze_instance.add_thing(agent, start)
maze_instance.add_thing(goal, end)
maze_instance.run(len(steps))
# print(path_length,time,space)


38 201 9
